In [1]:
import tensorflow as tf
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
import speech_recognition as sr
import datetime
import pytz


In [2]:
model_name = "Helsinki-NLP/opus-mt-en-hi"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_name)

C:\Users\swarnadeep acharya\anaconda3\Lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


C:\Users\swarnadeep acharya\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\swarnadeep acharya\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [3]:
def is_time_after_6pm():
    # Get current time in IST
    ist = pytz.timezone('Asia/Kolkata')
    current_time = datetime.datetime.now(ist)
    return current_time.hour >= 18


In [10]:
def translate_text(text):
    # Tokenize and translate
    input_ids = tokenizer(text, return_tensors="tf").input_ids
    translated_ids = model.generate(input_ids, num_beams=3, max_length=50, early_stopping=True)
    translated_text = tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    return translated_text

In [11]:
def recognize_speech():
    recognizer = sr.Recognizer()
    microphone = sr.Microphone()

    with microphone as source:
        print("Please speak into the microphone...")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)

    try:
        text = recognizer.recognize_google(audio)
        print(f"Recognized text: {text}")
        return text
    except sr.UnknownValueError:
        print("Sorry, I did not understand that. Could you please repeat?")
        return None
    except sr.RequestError:
        print("Could not request results; check your network connection")
        return None



In [14]:
def main():
    if not is_time_after_6pm():
        print("Please try after 6 PM IST")
        return
    
    text = None
    while text is None:
        text = recognize_speech()
    
    if text.strip().lower().startswith(('m', 'o')):
        print("Translation not allowed for text starting with 'M' or 'O'.")
        return

    translated_text = translate_text(text)
    print(f"Translated text: {translated_text}")

In [15]:
if __name__ == "__main__":
    main()

Please speak into the microphone...
Recognized text: my name is
Translation not allowed for text starting with 'M' or 'O'.
